In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("pyspark1").master('local[*]').getOrCreate()
sc = spark.sparkContext

22/10/05 06:00:36 WARN Utils: Your hostname, sunghwanui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.16.227.180 instead (on interface en0)
22/10/05 06:00:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/05 06:00:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [66]:
def capital(str):
    result = ""
#     words = str.spilt(" ")
    for ward in words:
        result = result + word[0:1].upper() + word[1:]
    return result

In [8]:
%%writefile test5.txt
good morning
danny
I love you danny
how are you?
are you good?
good good good

Writing test5.txt


In [17]:
import os

temp = os.path.abspath('./test5.txt')
data = sc.textFile(temp)
data.collect()

['good morning',
 'danny',
 'I love you danny',
 'how are you?',
 'are you good?',
 'good good good']

In [20]:
data.map(lambda x : x.upper()).collect()

['GOOD MORNING',
 'DANNY',
 'I LOVE YOU DANNY',
 'HOW ARE YOU?',
 'ARE YOU GOOD?',
 'GOOD GOOD GOOD']

In [34]:
data = sc.textFile('./sample.txt')
df = data.flatMap(lambda x : x.split(" ")).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).collect()
df = spark.createDataFrame(df, schema=['word', 'count'])
df.show()


+--------------------+-----+
|                word|count|
+--------------------+-----+
|                good|    1|
|                    | 5219|
|             General|    1|
|            Commands|    1|
|              Manual|    1|
|             FIND(1)|    1|
|    f\bfi\bin\bnd\bd|   38|
|                walk|    1|
|                   ||   10|
|            -\b-L\bL|    7|
|           -\b-P\bP]|    2|
|           [-\b-f\bf|    1|
|    _\bp_\ba_\bt_\bh|    3|
|        _\b._\b._\b.|    1|
|[_\be_\bx_\bp_\br...|    2|
|            -\b-f\bf|    2|
|                 The|   41|
|           directory|   12|
|                tree|    2|
|             listed,|    1|
+--------------------+-----+
only showing top 20 rows



In [50]:
df = df.select('count','word').orderBy(col('count').desc())
df.show()

+-----+----------------+
|count|            word|
+-----+----------------+
| 5219|                |
|  271|             the|
|  106|              is|
|  100|              to|
|   76|              of|
|   68|               a|
|   66|            file|
|   60|             and|
|   58|              if|
|   49|              in|
|   43|             are|
|   41|              be|
|   41|             The|
|   40|         primary|
|   39|              as|
|   39|            time|
|   38|f\bfi\bin\bnd\bd|
|   33|              by|
|   32|             not|
|   30|            _\bn|
+-----+----------------+
only showing top 20 rows



In [44]:
remove = df.first()[0]

In [64]:
df.filter((col("count") == "1") & (col("word") =="found")).show()

+-----+-----+
|count| word|
+-----+-----+
|    1|found|
+-----+-----+



In [74]:
simpleData = [("james hon","Sales","NY",90000,34,10000), \
    ("michael jj","Sales","NY",86000,56,20000), \
    ("robert rt","Sales","CA",81000,30,23000), \
    ("daria","Finance","CA",90000,24,23000), \
    ("aaman","Finance","CA",99000,40,24000), \
    ("ccott","Finance","NY",83000,36,19000), \
    ("hen","Finance","NY",79000,53,15000), \
    ("deff  ll","Marketing","CA",80000,25,18000), \
    ("aumar  mo","Marketing","NY",91000,50,21000) \
  ]

In [75]:
columns = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(simpleData, schema=columns)
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|    james hon|     Sales|   NY| 90000| 34|10000|
|   michael jj|     Sales|   NY| 86000| 56|20000|
|    robert rt|     Sales|   CA| 81000| 30|23000|
|        daria|   Finance|   CA| 90000| 24|23000|
|        aaman|   Finance|   CA| 99000| 40|24000|
|        ccott|   Finance|   NY| 83000| 36|19000|
|          hen|   Finance|   NY| 79000| 53|15000|
|     deff  ll| Marketing|   CA| 80000| 25|18000|
|    aumar  mo| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [133]:
df.withColumn("double salary", lit(col("salary") * 2)).show()

+-------------+----------+-----+------+---+-----+-------------+
|employee_name|department|state|salary|age|bonus|double salary|
+-------------+----------+-----+------+---+-----+-------------+
|    james hon|     Sales|   NY| 90000| 34|10000|       180000|
|   michael jj|     Sales|   NY| 86000| 56|20000|       172000|
|    robert rt|     Sales|   CA| 81000| 30|23000|       162000|
|        daria|   Finance|   CA| 90000| 24|23000|       180000|
|        aaman|   Finance|   CA| 99000| 40|24000|       198000|
|        ccott|   Finance|   NY| 83000| 36|19000|       166000|
|          hen|   Finance|   NY| 79000| 53|15000|       158000|
|     deff  ll| Marketing|   CA| 80000| 25|18000|       160000|
|    aumar  mo| Marketing|   NY| 91000| 50|21000|       182000|
+-------------+----------+-----+------+---+-----+-------------+



In [152]:
from pyspark.sql.functions import expr

new_column1 = expr("""
IF (state = 'NY', 1, IF(department = 'Sales',2,0))""")


In [153]:
df.withColumn("new1", new_column1).show()

+-------------+----------+-----+------+---+-----+----+
|employee_name|department|state|salary|age|bonus|new1|
+-------------+----------+-----+------+---+-----+----+
|    james hon|     Sales|   NY| 90000| 34|10000|   1|
|   michael jj|     Sales|   NY| 86000| 56|20000|   1|
|    robert rt|     Sales|   CA| 81000| 30|23000|   2|
|        daria|   Finance|   CA| 90000| 24|23000|   0|
|        aaman|   Finance|   CA| 99000| 40|24000|   0|
|        ccott|   Finance|   NY| 83000| 36|19000|   1|
|          hen|   Finance|   NY| 79000| 53|15000|   1|
|     deff  ll| Marketing|   CA| 80000| 25|18000|   0|
|    aumar  mo| Marketing|   NY| 91000| 50|21000|   1|
+-------------+----------+-----+------+---+-----+----+



In [159]:
from pyspark.sql.functions import when

new_column2= when(col('state') == 'NY',1)\
            .when(col('department')=='Sales',2)\
            .otherwise(0)

In [160]:
df.withColumn('new2', new_column2).show()

+-------------+----------+-----+------+---+-----+----+
|employee_name|department|state|salary|age|bonus|new2|
+-------------+----------+-----+------+---+-----+----+
|    james hon|     Sales|   NY| 90000| 34|10000|   1|
|   michael jj|     Sales|   NY| 86000| 56|20000|   1|
|    robert rt|     Sales|   CA| 81000| 30|23000|   2|
|        daria|   Finance|   CA| 90000| 24|23000|   0|
|        aaman|   Finance|   CA| 99000| 40|24000|   0|
|        ccott|   Finance|   NY| 83000| 36|19000|   1|
|          hen|   Finance|   NY| 79000| 53|15000|   1|
|     deff  ll| Marketing|   CA| 80000| 25|18000|   0|
|    aumar  mo| Marketing|   NY| 91000| 50|21000|   1|
+-------------+----------+-----+------+---+-----+----+



In [161]:
df.withColumn("new3", lit(3)).show()

+-------------+----------+-----+------+---+-----+----+
|employee_name|department|state|salary|age|bonus|new3|
+-------------+----------+-----+------+---+-----+----+
|    james hon|     Sales|   NY| 90000| 34|10000|   3|
|   michael jj|     Sales|   NY| 86000| 56|20000|   3|
|    robert rt|     Sales|   CA| 81000| 30|23000|   3|
|        daria|   Finance|   CA| 90000| 24|23000|   3|
|        aaman|   Finance|   CA| 99000| 40|24000|   3|
|        ccott|   Finance|   NY| 83000| 36|19000|   3|
|          hen|   Finance|   NY| 79000| 53|15000|   3|
|     deff  ll| Marketing|   CA| 80000| 25|18000|   3|
|    aumar  mo| Marketing|   NY| 91000| 50|21000|   3|
+-------------+----------+-----+------+---+-----+----+



In [165]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = sc.parallelize(dept)
rdd.collect()

[('Finance', 10), ('Marketing', 20), ('Sales', 30), ('IT', 40)]

In [166]:
df = rdd.toDF()
df.show()

+---------+---+
|       _1| _2|
+---------+---+
|  Finance| 10|
|Marketing| 20|
|    Sales| 30|
|       IT| 40|
+---------+---+



In [167]:
df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [168]:
deptColumns = ["dept_name","dept_id"]
df = rdd.toDF(deptColumns)
df.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [169]:
df.printSchema()

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)



In [170]:
df = spark.createDataFrame(rdd, deptColumns)
df.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [171]:
df.printSchema()

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)



In [179]:
df.foreach(lambda x : 

Row(dept_name='Sales', dept_id=30)
Row(dept_name='Marketing', dept_id=20)
Row(dept_name='Finance', dept_id=10)
Row(dept_name='IT', dept_id=40)


In [185]:
df.foreach(print)

Row(dept_name='IT', dept_id=40)
Row(dept_name='Finance', dept_id=10)
Row(dept_name='Sales', dept_id=30)
Row(dept_name='Marketing', dept_id=20)


In [193]:
a = df.foreach(lambda x: print(type(x)))


<class 'pyspark.sql.types.Row'>
<class 'pyspark.sql.types.Row'>
<class 'pyspark.sql.types.Row'>
<class 'pyspark.sql.types.Row'>


In [86]:
def capital(x):
    result = ""
    x = x.split(" ")
    for word in x:
        result = result + word[0:1].upper() + word[1:]
    return result
    

In [87]:
from pyspark.sql.functions import udf

capUDF = udf(lambda x : capital(x))


In [89]:
df.select(capUDF(col("employee_name")).alias("employee_name"), col("department")).show()

+-------------+----------+
|employee_name|department|
+-------------+----------+
|     JamesHon|     Sales|
|    MichaelJj|     Sales|
|     RobertRt|     Sales|
|        Daria|   Finance|
|        Aaman|   Finance|
|        Ccott|   Finance|
|          Hen|   Finance|
|       DeffLl| Marketing|
|      AumarMo| Marketing|
+-------------+----------+



In [98]:
from pyspark.sql.functions import concat,lit

df.select(concat(col("employee_name"),lit(' - '), col("department"))).show()

+--------------------------------------+
|concat(employee_name,  - , department)|
+--------------------------------------+
|                     james hon - Sales|
|                    michael jj - Sales|
|                     robert rt - Sales|
|                       daria - Finance|
|                       aaman - Finance|
|                       ccott - Finance|
|                         hen - Finance|
|                  deff  ll - Marketing|
|                  aumar  mo - Marke...|
+--------------------------------------+



In [105]:
from pyspark.sql.functions import concat_ws
df.select(concat_ws(",",col("employee_name"),col("department")).alias("name")).show()

+-------------------+
|               name|
+-------------------+
|    james hon,Sales|
|   michael jj,Sales|
|    robert rt,Sales|
|      daria,Finance|
|      aaman,Finance|
|      ccott,Finance|
|        hen,Finance|
| deff  ll,Marketing|
|aumar  mo,Marketing|
+-------------------+



In [108]:
df.select(concat(col("employee_name"),lit(" good guy")).alias("name"), lit(col("salary")*1000).alias("double salary") ).show()

+-------------------+-------------+
|               name|double salary|
+-------------------+-------------+
| james hon good guy|     90000000|
|michael jj good guy|     86000000|
| robert rt good guy|     81000000|
|     daria good guy|     90000000|
|     aaman good guy|     99000000|
|     ccott good guy|     83000000|
|       hen good guy|     79000000|
|  deff  ll good guy|     80000000|
| aumar  mo good guy|     91000000|
+-------------------+-------------+



In [111]:
df.rdd.map(lambda x: x[0]).collect()

['james hon',
 'michael jj',
 'robert rt',
 'daria',
 'aaman',
 'ccott',
 'hen',
 'deff  ll',
 'aumar  mo']

In [122]:
rdd = df.rdd.map(lambda x: x["employee_name"]+" , "+x["department"])

In [131]:
rdd.collect()
rdd.map(lambda x : (x,)).toDF().show()
# rdd.toDF(["name","df"])

+--------------------+
|                  _1|
+--------------------+
|   james hon , Sales|
|  michael jj , Sales|
|   robert rt , Sales|
|     daria , Finance|
|     aaman , Finance|
|     ccott , Finance|
|       hen , Finance|
|deff  ll , Marketing|
|aumar  mo , Marke...|
+--------------------+



In [132]:
rdd.map(lambda x : (x,)).collect()

[('james hon , Sales',),
 ('michael jj , Sales',),
 ('robert rt , Sales',),
 ('daria , Finance',),
 ('aaman , Finance',),
 ('ccott , Finance',),
 ('hen , Finance',),
 ('deff  ll , Marketing',),
 ('aumar  mo , Marketing',)]

In [91]:
df.select(col('employee_name').map(capital)).show()

TypeError: 'Column' object is not callable

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

22/10/04 06:04:21 WARN Utils: Your hostname, sunghwanui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.16.227.180 instead (on interface en0)
22/10/04 06:04:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/04 06:04:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc = spark.sparkContext
sc.getConf().getAll()

[('spark.app.startTime', '1664881461943'),
 ('spark.driver.host', '172.16.227.180'),
 ('spark.sql.warehouse.dir',
  'file:/Users/sunghwanki/Desktop/Project/Python_Advance/Pyspark/spark-warehouse'),
 ('spark.app.name', 'test'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '51419'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1664881463505'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
rdd = sc.range(1,5)
rdd.collect()

[1, 2, 3, 4]

In [8]:
print(sc.uiWebUrl)
print(sc.applicationId)
print(sc.version)

http://172.16.227.180:4040
local-1664881463505
3.2.1


In [10]:
!pwd

/Users/sunghwanki/Desktop/Project/Python_Advance/Pyspark


In [21]:
import os
# tempdir = os.path.abspath('./sample.txt')
tempdir = os.path.join('/Users/sunghwanki/Desktop/Project/Python_Advance/Pyspark','sample.txt')
print(tempdir)
with open(tempdir, 'w') as f:
    f.write("good evening")
    

rdd = sc.textFile('./sample.txt')
rdd.collect()


/Users/sunghwanki/Desktop/Project/Python_Advance/Pyspark/sample.txt


['good evening']

In [22]:
!man find >> ./sample.txt
rdd = sc.textFile('./sample.txt')
rdd.top(10)

['macOS 12.5                       March 13, 2021                       macOS 12.5',
 'good eveningFIND(1)                      General Commands Manual                     FIND(1)',
 'S\x08SY\x08YN\x08NO\x08OP\x08PS\x08SI\x08IS\x08S',
 'S\x08ST\x08TA\x08AN\x08ND\x08DA\x08AR\x08RD\x08DS\x08S',
 'S\x08SE\x08EE\x08E A\x08AL\x08LS\x08SO\x08O',
 'P\x08PR\x08RI\x08IM\x08MA\x08AR\x08RI\x08IE\x08ES\x08S',
 'O\x08OP\x08PE\x08ER\x08RA\x08AT\x08TO\x08OR\x08RS\x08S',
 'N\x08NA\x08AM\x08ME\x08E',
 'H\x08HI\x08IS\x08ST\x08TO\x08OR\x08RY\x08Y',
 'E\x08EX\x08XA\x08AM\x08MP\x08PL\x08LE\x08ES\x08S']

In [30]:
myList = [i for i in range(1,100) if i % 2 == 0]
print(myList)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98]


In [31]:
rdd = sc.parallelize(myList)
print(rdd.collect())

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98]


In [32]:
%%writefile ./sample1.txt
my name is good
good name is my name
how are you?
okay?
good

Writing ./sample1.txt


In [39]:
myRdd = sc.textFile(os.path.abspath('./sample1.txt'))
myRdd = myRdd.flatMap(lambda x: x.split(" ")).map(lambda word:(word,1)).reduceByKey(lambda x,y: x+y)
myRdd.collect()
                    

[('name', 3),
 ('is', 2),
 ('good', 3),
 ('are', 1),
 ('you?', 1),
 ('my', 2),
 ('how', 1),
 ('okay?', 1)]

In [42]:
output = myRdd.collect()
for (word,count) in output:
    print("%s: %i"% (word, count))

name: 3
is: 2
good: 3
are: 1
you?: 1
my: 2
how: 1
okay?: 1


In [43]:
myRdd.first()

('name', 3)

In [44]:
myRdd.take(3)

[('name', 3), ('is', 2), ('good', 3)]

In [53]:
%%writefile ./test.txt
good morning
this is new day
I love so much
I am good
you are sunset
morning
good 
bad
sunset
sunrise

Overwriting ./test.txt


In [57]:
rdd = sc.textFile(os.path.abspath('./test.txt'))
rdd = rdd.flatMap(lambda x : x.split(" ")).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
mylist = rdd.collect()
for a,b in mylist:
    print('%s : %i'%(a,b))

good : 3
this : 1
is : 1
new : 1
love : 1
am : 1
are : 1
sunset : 2
 : 1
morning : 2
day : 1
I : 2
so : 1
much : 1
you : 1
bad : 1
sunrise : 1


In [ ]:
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd = sc.parallelize(dataList)



In [69]:
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Raman","Finance","CA",99000,40,24000), \
    ("Scott","Finance","NY",83000,36,19000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]

In [70]:
columns = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(simpleData, schema=columns)
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)



In [71]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [72]:
df.show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [73]:
df.sort("department", "salary").show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        James|     Sales|   NY| 90000| 34|10000|
+-------------+----------+-----+------+---+-----+



In [77]:
df.orderBy("department","salary").show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        James|     Sales|   NY| 90000| 34|10000|
+-------------+----------+-----+------+---+-----+



In [76]:
from pyspark.sql.functions import col
df.sort(col("state"), col("salary").desc()).show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
+-------------+----------+-----+------+---+-----+



In [78]:
df.orderBy(col("state"), col("salary").desc()).show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
+-------------+----------+-----+------+---+-----+



In [82]:
df.createOrReplaceTempView("EMP")
spark.sql("select * from EMP order by state, salary desc").show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
+-------------+----------+-----+------+---+-----+



In [83]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]

df = spark.createDataFrame(arrayData)
df.show()

+----------+-------------------+--------------------+
|        _1|                 _2|                  _3|
+----------+-------------------+--------------------+
|     James|      [Java, Scala]|{eye -> brown, ha...|
|   Michael|[Spark, Java, null]|{eye -> null, hai...|
|    Robert|         [CSharp, ]|{eye -> , hair ->...|
|Washington|               null|                null|
| Jefferson|             [1, 2]|                  {}|
+----------+-------------------+--------------------+



In [84]:
df = spark.createDataFrame(arrayData, schema = ['name', 'knowledge', 'property'])
df.show()

+----------+-------------------+--------------------+
|      name|          knowledge|            property|
+----------+-------------------+--------------------+
|     James|      [Java, Scala]|{eye -> brown, ha...|
|   Michael|[Spark, Java, null]|{eye -> null, hai...|
|    Robert|         [CSharp, ]|{eye -> , hair ->...|
|Washington|               null|                null|
| Jefferson|             [1, 2]|                  {}|
+----------+-------------------+--------------------+



In [85]:
df.collect()

[Row(name='James', knowledge=['Java', 'Scala'], property={'eye': 'brown', 'hair': 'black'}),
 Row(name='Michael', knowledge=['Spark', 'Java', None], property={'eye': None, 'hair': 'brown'}),
 Row(name='Robert', knowledge=['CSharp', ''], property={'eye': '', 'hair': 'red'}),
 Row(name='Washington', knowledge=None, property=None),
 Row(name='Jefferson', knowledge=['1', '2'], property={})]

In [88]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- knowledge: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- property: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [90]:
from pyspark.sql.functions import explode
df2 = df.select("name", explode("knowledge"))
df2.show()

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  null|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [93]:
words_list = sc.parallelize (
  ["pyspark", 
  "interview", 
  "questions", 
  "at", 
  "interviewbit"]
)

filtered = words_list.filter(lambda x : 'interview' in x)
filtered.collect()

['interview', 'interviewbit']

In [94]:
filtered.count()

2

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf

sc = SparkContext(master='local', appName='pyspark-test')
sc


22/10/03 06:33:58 WARN Utils: Your hostname, sunghwanui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.16.227.180 instead (on interface en0)
22/10/03 06:33:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/03 06:33:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=pyspark-test>

In [2]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '49503'),
 ('spark.driver.host', '172.16.227.180'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-test'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1664796841274'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1664796839628')]

In [7]:
sc.stop()

In [6]:
conf = SparkConf()
conf.setAppName("test").setMaster("local")

sc = SparkContext(conf=conf)
sc

<SparkContext master=local appName=test>

In [54]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("session_test").master('local[*]').getOrCreate()
spark

22/10/03 07:18:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [55]:
spark.sparkContext

<SparkContext master=local[*] appName=session_test>

In [10]:
spark.sparkContext.appName

'session_test'

In [11]:
sc = spark.sparkContext

In [13]:
rdd = sc.range(1,5)

In [14]:
rdd.collect()

[1, 2, 3, 4]

In [15]:
sc.applicationId

'local-1664797151878'

In [16]:
sc.uiWebUrl

'http://172.16.227.180:4040'

In [17]:
sc.version

'3.2.1'

In [20]:
import os
path = os.path.abspath('./sample_1.txt')

with open(path, 'w') as f:
    f.write("good morning")



In [22]:
print(path)

/Users/sunghwanki/Desktop/Project/Python_Advance/Pyspark/sample_1.txt


In [24]:
rdd = sc.textFile('./sample_1.txt')
rdd.collect()

['good morning']

In [25]:
myList=[1,2,3,4,5]
myRdd = sc.parallelize(myList)
myRdd

ParallelCollectionRDD[9] at readRDDFromFile at PythonRDD.scala:274

In [27]:
myRdd.first()

1

In [28]:
myRdd.take(3)

[1, 2, 3]

In [29]:
myRdd.collect()

[1, 2, 3, 4, 5]

In [31]:
%%writefile ./sample_2.txt
hi my name is danny
nice meet you
good morning
good evening
the king of kings
do you know me?
gogo

Writing ./sample_2.txt


In [32]:
!cat ./sample_2.txt

hi my name is danny
nice meet you
good morning
good evening
the king of kings
do you know me?
gogo


In [33]:
myRdd = sc.textFile('./sample_2.txt')
myRdd

./sample_2.txt MapPartitionsRDD[15] at textFile at NativeMethodAccessorImpl.java:0

In [34]:
myRdd.first()

'hi my name is danny'

In [36]:
myRdd.top(3)

['the king of kings', 'nice meet you', 'hi my name is danny']

In [37]:
myRdd.collect()

['hi my name is danny',
 'nice meet you',
 'good morning',
 'good evening',
 'the king of kings',
 'do you know me?',
 'gogo']

In [38]:
myRdd.count()

7

In [39]:
myRdd.take(5)

['hi my name is danny',
 'nice meet you',
 'good morning',
 'good evening',
 'the king of kings']

In [42]:
myRdd1=[a for a in range(20)]
myRdd2= sc.parallelize(myRdd1)

In [43]:
myRdd2.count()

20

In [44]:
myRdd2.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [45]:
type(myRdd2)

pyspark.rdd.RDD

In [46]:
def sub(n):
    return n-1

subRdd = myRdd2.map(sub)
subRdd

PythonRDD[23] at RDD at PythonRDD.scala:53

In [47]:
subRdd.collect()

[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [48]:
data = ["Project Gutenberg’s",
        "Alice’s Adventures in Wonderland",
        "Project Gutenberg’s",
        "Adventures in Wonderland",
        "Project Gutenberg’s"]

rdd = sc.parallelize(data)
for i in rdd.collect():
    print(i)

Project Gutenberg’s
Alice’s Adventures in Wonderland
Project Gutenberg’s
Adventures in Wonderland
Project Gutenberg’s


In [49]:
rdd.collect()

['Project Gutenberg’s',
 'Alice’s Adventures in Wonderland',
 'Project Gutenberg’s',
 'Adventures in Wonderland',
 'Project Gutenberg’s']

In [50]:
def sub2(n):
    return n.split(" ")

In [51]:
rdd2 = rdd.flatMap(sub2)
rdd2.collect()

['Project',
 'Gutenberg’s',
 'Alice’s',
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 'Gutenberg’s',
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 'Gutenberg’s']

In [52]:
rdd3 = rdd.flatMap(lambda x:x.split(" "))
rdd3.collect()

['Project',
 'Gutenberg’s',
 'Alice’s',
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 'Gutenberg’s',
 'Adventures',
 'in',
 'Wonderland',
 'Project',
 'Gutenberg’s']

In [53]:
sc.stop()

In [59]:
spark

In [62]:
sc = spark.sparkContext

In [63]:
words_list = sc.parallelize (
  ["pyspark", 
  "interview", 
  "questions", 
  "at", 
  "interviewbit"]
)

filtered_words = words_list.filter(lambda x: 'interview' in x)
filtered_words.collect()

['interview', 'interviewbit']

In [64]:
filtered_words.count()

2

In [72]:
filtered_words.collect()[0] = 'data'

In [73]:
filtered_words.collect()

['interview', 'interviewbit']